In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.svm import SVC
gender_lb = LabelBinarizer()
married_le = LabelEncoder()
edu_le = LabelEncoder()
se_le = LabelEncoder()
parea_lb = LabelBinarizer()
ls_lp=LabelEncoder()

In [2]:
def impute_data(loan_data):
    loan_data.Gender=loan_data.Gender.fillna('Unknown')
    loan_data.Married=loan_data.Married.fillna('Yes')#Mostly married people are the once to apply for 
    loan_data.Dependents=loan_data.Dependents.replace('3+','3').fillna('2')#Considering Hum do Humare do.
    loan_data.Self_Employed=loan_data.Self_Employed.fillna('No')#considering most ppl applying for loan are salaried.
    loan_data.loc[pd.isnull(loan_data.LoanAmount),'LoanAmount']=((loan_data.ApplicantIncome+loan_data.CoapplicantIncome)/1000)*60#Amount which applied most number of times
    loan_data.loc[pd.isnull(loan_data.Loan_Amount_Term),'Loan_Amount_Term']=round((loan_data.LoanAmount*1000)/((loan_data.ApplicantIncome+loan_data.CoapplicantIncome)*0.15))#Considering around 20% of income will be spend on emi
    loan_data.Credit_History=loan_data.Credit_History.fillna(1.0)#be positive trust the customer
    return loan_data

In [3]:
def transform_data(loan_data,rev=False):
    global loan_data_loan_id
    if rev==False:
        loan_data=pd.concat([loan_data,pd.DataFrame(gender_lb.fit_transform(loan_data.Gender),columns=gender_lb.__dict__['classes_']).reset_index(drop=True)],axis=1)
        loan_data.Married=married_le.fit_transform(loan_data.Married)
        loan_data.Education=edu_le.fit_transform(loan_data.Education)
        if loan_data.columns.contains('Loan_Status'):
            loan_data.Loan_Status=ls_lp.fit_transform(loan_data.Loan_Status)
        loan_data.Self_Employed=se_le.fit_transform(loan_data.Self_Employed)
        loan_data=pd.concat([loan_data,pd.DataFrame(parea_lb.fit_transform(loan_data.Property_Area),columns=parea_lb.__dict__['classes_']).reset_index(drop=True)],axis=1)
        loan_data_loan_id=loan_data['Loan_ID']
        loan_data.drop(['Property_Area','Gender','Loan_ID'],axis=1,inplace=True)
    else:
        loan_data=pd.concat([loan_data,pd.DataFrame(gender_lb.inverse_transform(loan_data[gender_lb.__dict__['classes_']].values),columns=['Gender']).reset_index(drop=True)],axis=1)
        loan_data.Married=married_le.inverse_transform(loan_data.Married)
        loan_data.Education=edu_le.inverse_transform(loan_data.Education)
        loan_data.Loan_Status=ls_lp.inverse_transform(loan_data.Loan_Status)
        loan_data.Self_Employed=se_le.inverse_transform(loan_data.Self_Employed)
        loan_data=pd.concat([loan_data,pd.DataFrame(parea_lb.inverse_transform(loan_data[parea_lb.__dict__['classes_']].values),columns=['Property_Area']).reset_index(drop=True)],axis=1)
        loan_data.drop(gender_lb.__dict__['classes_'].tolist()+parea_lb.__dict__['classes_'].tolist(),axis=1,inplace=True)
        loan_data=pd.concat([loan_data,pd.DataFrame(loan_data_loan_id,columns=['Loan_ID']).reset_index(drop=True)],axis=1)
    return loan_data

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


def svc_param_grid(X, y, nfolds):
    param_grid = {'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
                  'C': np.arange(0.5,1.5,0.1),
                  'gamma': np.arange(0.001,0.030,0.001)#,
                  #'max_iter':[3]
                 }
    
    grid_search = GridSearchCV(SVC(), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    print(grid_search.best_params_)
    
    print("Detailed classification report:")
    print()
    y_true, y_pred = y, grid_search.predict(X)
    print(classification_report(y_true, y_pred) , confusion_matrix(y_true,y_pred))
    print()
    
    return grid_search.best_params_,grid_search.score(X,y)

In [5]:
loan_data_train = pd.read_csv('train_u6lujuX_CVtuZ9i.csv')

In [6]:
loan_data_train=impute_data(loan_data_train)
loan_data_train=transform_data(loan_data_train)

In [7]:
X = loan_data_train.drop(['Loan_Status'],axis=1)
y = loan_data_train['Loan_Status']
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_std = ss.fit_transform(X)
# from sklearn.preprocessing import PolynomialFeatures
# pf=PolynomialFeatures(2)
# X_Poly=pf.fit_transform(X_std)
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X_std,y,test_size=0.25,random_state=0)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [ ]:
# svc_param_grid(loan_data_train.drop(['Loan_Status'],axis=1),loan_data_train['Loan_Status'],10)
svc_param_grid(X,y,10)

In [64]:
svc = SVC(C= 0.5, gamma= 0.023, kernel= 'sigmoid')
svc.fit(xtrain,ytrain)
preds = svc.predict(xtrain)
testpreds = svc.predict(xtest)
# predp = svc.predict_proba(xtrain)
svc.score(xtest,ytest)

0.8311688311688312

In [25]:
C = [0.5,1,1.5,2,3,5,6,7,8,9,10,12,15]
# Ker=
df=pd.DataFrame(columns=['Parameter','train_score','test_score'])
train_score = []
test_score = []
for c in np.arange(0.001,0.030,0.001) :
    svc = SVC(C=1,gamma=c)
    svc.fit(xtrain,ytrain)
    df=df.append(pd.DataFrame({'Parameter':[c],'train_score':[svc.score(xtrain,ytrain)],'test_score':[svc.score(xtest,ytest)]}),ignore_index=True)
#     test_score.append()
#     train_score.append()
#     print("Running for ",c)
df

,Parameter,train_score,test_score
0,0.001,0.676087,0.720779
1,0.002,0.802174,0.831169
2,0.003,0.802174,0.831169
3,0.004,0.804348,0.831169
4,0.005,0.804348,0.831169
5,0.006,0.804348,0.831169
6,0.007,0.804348,0.837662
7,0.008,0.804348,0.837662
8,0.009,0.804348,0.837662
9,0.010,0.804348,0.837662


In [65]:
from sklearn.metrics import accuracy_score , recall_score, precision_score , roc_auc_score
pd.DataFrame(data=[[accuracy_score(ytrain, preds), recall_score(ytrain, preds),precision_score(ytrain, preds), roc_auc_score(ytrain, preds)],
                   [accuracy_score(ytest, testpreds), recall_score(ytest, testpreds),precision_score(ytest, testpreds), roc_auc_score(ytest, testpreds)]],
             columns=["accuracy", "recall", "precision", "roc_auc_score"], index=['trainScores','testScores'])

,accuracy,recall,precision,roc_auc_score
trainScores,0.797826,0.977492,0.779487,0.700155
testScores,0.831169,0.981982,0.819549,0.711921


In [66]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(ytrain,preds))
print(confusion_matrix(ytest,testpreds))

[[ 63  86]
 [  7 304]]
[[ 19  24]
 [  2 109]]


In [69]:
from sklearn.model_selection import cross_val_score
score_train= svc.score(xtrain, ytrain)
score_val = cross_val_score(svc,xtrain, ytrain, cv=10).mean()
print('training score = %s , while validation score = %s' %(score_train , score_val))

training score = 0.7978260869565217 , while validation score = 0.8022962277726385


In [490]:
from sklearn.metrics import accuracy_score , recall_score, precision_score , roc_auc_score

for THRESHOLD in range(1,100) :
    preds = np.where(logreg.predict_proba(xtrain)[:,1] > THRESHOLD/100, 1, 0)
    df = pd.DataFrame(data=np.array([THRESHOLD,accuracy_score(ytrain, preds), recall_score(ytrain, preds),
                       precision_score(ytrain, preds), roc_auc_score(ytrain, preds)]).reshape(1,-1), 
                 columns=["THRESHOLD","accuracy", "recall", "precision", "roc_auc_score"])
    print(df)

   THRESHOLD  accuracy  recall  precision  roc_auc_score
0        1.0  0.676087     1.0   0.676087            0.5
   THRESHOLD  accuracy  recall  precision  roc_auc_score
0        2.0  0.676087     1.0   0.676087            0.5
   THRESHOLD  accuracy  recall  precision  roc_auc_score
0        3.0  0.682609     1.0   0.680525       0.510067
   THRESHOLD  accuracy  recall  precision  roc_auc_score
0        4.0       0.7     1.0    0.69265       0.536913
   THRESHOLD  accuracy  recall  precision  roc_auc_score
0        5.0  0.713043     1.0   0.702032       0.557047
   THRESHOLD  accuracy    recall  precision  roc_auc_score
0        6.0  0.726087  0.996785   0.712644       0.578929
   THRESHOLD  accuracy    recall  precision  roc_auc_score
0        7.0      0.75  0.996785   0.731132       0.615842
   THRESHOLD  accuracy    recall  precision  roc_auc_score
0        8.0  0.756522  0.993569    0.73747       0.627657
   THRESHOLD  accuracy    recall  precision  roc_auc_score
0        9.0  0.7

   THRESHOLD  accuracy   recall  precision  roc_auc_score
0       73.0  0.730435  0.77492   0.816949       0.706252
   THRESHOLD  accuracy    recall  precision  roc_auc_score
0       74.0   0.71087  0.733119   0.820144       0.698774
   THRESHOLD  accuracy   recall  precision  roc_auc_score
0       75.0  0.697826  0.70418   0.823308       0.694372
   THRESHOLD  accuracy    recall  precision  roc_auc_score
0       76.0  0.682609  0.672026   0.826087       0.688362
   THRESHOLD  accuracy    recall  precision  roc_auc_score
0       77.0  0.667391  0.630225   0.837607       0.687596
   THRESHOLD  accuracy    recall  precision  roc_auc_score
0       78.0  0.654348  0.588424    0.85514       0.690185
   THRESHOLD  accuracy    recall  precision  roc_auc_score
0       79.0  0.628261  0.533762   0.864583       0.679633
   THRESHOLD  accuracy    recall  precision  roc_auc_score
0       80.0  0.602174  0.488746   0.863636       0.663836
   THRESHOLD  accuracy    recall  precision  roc_auc_score
0

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: U

In [73]:
loan_data_test=pd.read_csv('test_Y3wMUE5_7gLdaTN.csv')

In [74]:
loan_data_test=impute_data(loan_data_test)

In [75]:
loan_data_test=impute_data(loan_data_test)
loan_data_test=transform_data(loan_data_test)
ss_test = StandardScaler()
X_std = ss_test.fit_transform(loan_data_test)
# pf_test=PolynomialFeatures(2)
# X_Poly=pf_test.fit_transform(X_std)
loan_data_test['Loan_Status']=svc.predict(X_std)
loan_data_test=transform_data(loan_data_test,rev=True)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [76]:
# pd.DataFrame([loan_data_loan_id,test_preds],columns=['Loan_ID','Loan_Status'])
# pd.DataFrame({'Loan_ID':loan_data_loan_id.values,'Loan_Status':test_preds}).to_csv('Loan_Predicted2.csv',index=False)
# pd.DataFrame
loan_data_test[['Loan_ID','Loan_Status']].to_csv('Loan_Predicted3.csv',index=False)